# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


## Enabling and testing the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

Next, we'll confirm that we can connect to the GPU with tensorflow:

In [17]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


# Defining constants

In [18]:
image_size = 227
number_of_channels = 3
number_of_classes = 10
training_path = "/Data/TrainingData"
validation_path = "/Data/ValidationData"
test_path = "/Data/TestData"

# The main part of the project

In [54]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, Lambda, BatchNormalization
from keras.utils import  np_utils
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from keras.optimizers import SGD, Adam
import numpy as np
from os import listdir
import cv2
from keras import regularizers
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.datasets import mnist
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report


def load_cnn_model(model_path):
    model = create_nn_model()
    model.load_weights(model_path)
    return model


def predict(model, train_path, test_path):
    print("\n*** PREDICTING ON TEST DATA ***")
 
    print("\n   Loading images...")
    train_x, train_y = load_images(train_path)
    test_x, test_y = load_images(test_path)
    print("Test num: " + str(len(test_x)))
    print("   DONE! Images are loaded.")

    datagen = ImageDataGenerator(rescale=1 / 255.0)
    # calculate mean on training dataset
    datagen.fit(train_x)
    # prepare an iterators to scale images
    print("Mean: ", datagen.mean)
    print("Std: ", datagen.std)
    test_iterator = datagen.flow(test_x, test_y, batch_size=64)
    print("Mean: ", datagen.mean)
    print("Std: ", datagen.std)

  
    predictions = model.predict_generator(test_iterator, steps=len(test_iterator), verbose=0)

    predictions = np.argmax(predictions, axis=1)
    # print classification report
    # print(classification_report(test_iterator.classes, predictions,
	  #              target_names=test_iterator.class_indices.keys()))

    test_y = np.argmax(test_y, axis=1)

    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))
    print(test_y)
    print(predictions)
    print(accuracy_score(test_y, predictions))


def fit_model(epochs, batch_size):

    # load images
    print("\n\tLoading training images...")
    train_x, train_y = load_images(training_path)

    print("\n\tLoading validation images...")
    x_validation, y_validation = load_images(validation_path)
    
    # create generator to center images
    datagen = ImageDataGenerator(rescale=1 / 255.0)
    # calculate mean on training dataset
    datagen.fit(train_x)
    # prepare an iterators to scale images
    train_iterator = datagen.flow(train_x, train_y, batch_size=64)
    test_iterator = datagen.flow(x_validation, y_validation, batch_size=batch_size)

    print('Mean: ', datagen.mean)
    mean = datagen.mean
    print('Std: ', datagen.std)
    std = datagen.std
    print("Creating nn model ...")
    model = create_nn_model()
    print("Model is created")

    history = model.fit_generator(train_iterator, validation_data=test_iterator, steps_per_epoch=len(train_iterator), epochs=epochs)
    # evaluate model 
    _, acc = model.evaluate_generator(test_iterator, steps=len(test_iterator), verbose=0)
    print('Test Accuracy: %.3f' % (acc * 100))

    # save model weights and return history
    model.save_weights('network.h5')
    return history


def load_images(folder):
    x = []
    y = []
    
    for wbcType in listdir(folder):
        for imgName in listdir(folder + '/' + wbcType):
            image = cv2.imread(folder + '/' + wbcType + '/' + imgName)

            if image is not None:
                dim = (image_size, image_size)
                # konvertovanje iz BGR u RGB model boja (OpenCV ucita sliku kao BGR)
                img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                # konvertovanje u grayscale sliku
                # img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY) 

                resized = cv2.resize(img_rgb, dim, interpolation=cv2.INTER_AREA)
                res =  np.reshape(resized, (image_size, image_size, 3))

                image_array = np.asarray(res)
                x.append(image_array)
                y.append(wbcType)

    y_array = np.asarray(y)

    # encode y data
    encoder = LabelEncoder()
    encoder.fit(y_array)
    encoded_y = encoder.transform(y_array)
    y = np_utils.to_categorical(encoded_y)

     
    return np.asarray(x), y


def create_nn_model():
    # Instantiate an empty model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(image_size, image_size, number_of_channels), kernel_size=(11, 11),
                     strides=(4, 4), padding="valid"))
    model.add(Activation("relu"))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11, 11), strides=(1, 1), padding="valid",
                       kernel_regularizer=regularizers.l2(0.01),
                     bias_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding="valid",
                     
                     kernel_regularizer=regularizers.l2(0.01),
                     bias_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding="valid",
                     kernel_regularizer=regularizers.l2(0.01),
                     bias_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding="valid",
                     kernel_regularizer=regularizers.l2(0.01),
                     bias_regularizer=regularizers.l2(0.01)))
    model.add(Activation("relu"))
    # Max Pooling
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid"))

    # Passing it to a Fully Connected layer
    model.add(Flatten())
    # 1st Fully Connected Layer
    model.add(Dense(4096, input_shape=(image_size, image_size, number_of_channels)))
    model.add(Activation("relu"))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))

    # 2nd Fully Connected Layer
    model.add(Dense(4096))
    model.add(Activation("relu"))
    # Add Dropout
    model.add(Dropout(0.4))

    # 3rd Fully Connected Layer
    model.add(Dense(1000))
    model.add(Activation("relu"))
    # Add Dropout
    model.add(Dropout(0.4))

    # Output Layer
    model.add(Dense(number_of_classes)) 
    model.add(Activation("softmax"))

    model.summary()

    # Compile the model
    # sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
    # sgd = SGD(lr=0.1, decay=1e-6, momentum=0.0, nesterov=False)
    # print("sgd")
    # opt = SGD(lr=0.01)
    learning_rate = 0.001
    decay_rate = learning_rate / 100
    sgd = SGD(lr= 0.005, momentum=0.8, decay= decay_rate, nesterov=True)
    model.compile(loss='categorical_crossentropy',
                  optimizer=sgd,
                  metrics=['accuracy'])
    return model

def create_vgg_model():

  model = Sequential()
  # model.add(Lambda(lambda x: x / 127.5 - 1., input_shape=(image_size, image_size, number_of_channels,), output_shape=(image_size, image_size, number_of_channels,)))
  # First Convolotional Layer
  model.add(Conv2D(input_shape=(image_size, image_size, number_of_channels),filters=64,kernel_size=(3,3),padding="same", 
                   activation="relu"))
  # Second Convolutional Layer
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  # model.add(BatchNormalization())

  # Third Convolutional Layer
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  # model.add(BatchNormalization())

  # Fourth Convolutional Layer
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  # model.add(BatchNormalization())

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu",
                   kernel_regularizer=regularizers.l2(0.01)))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  # model.add(BatchNormalization())
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  # model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu",
                   kernel_regularizer=regularizers.l2(0.01)))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
  # model.add(BatchNormalization())

  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu", kernel_regularizer=regularizers.l2(0.01)))
  model.add(Dense(units=4096,activation="relu"))
  # Add Dropout
  model.add(Dropout(0.4))
  # Output Layer
  model.add(Dense(number_of_classes)) # Da li je ovo broj klasa?
  model.add(Activation("softmax"))

  # model.summary()

  # opt = Adam(lr=0.01)
  learning_rate = 0.01
  decay_rate = learning_rate / 100
  opt = SGD(lr= 0.007, momentum=0.8, decay= decay_rate, nesterov=True)
  model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
  return model

def create_acc_diagram(history):
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.savefig('accuracy.pdf')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()


def create_loss_diagram(history):
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.savefig('loss.pdf')
    plt.show()



# Main function with menu

In [ ]:
import cv2
import numpy as np
from os import listdir
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib


def menu():

    proceed = True
    while proceed:

       # print("\n1. Data Augmentation ")
        print("1. Train model")
        print("2. Predict")
        print("0. Exit")

        select = int(input("Please select an option: \n"))

        if select == 1:
            print("Calling model training ... ")
            with tf.device('/device:GPU:0'):
                history = fit_model(50, 128)
                print(history.history.keys())
                create_acc_diagram(history)
                create_loss_diagram(history)
        elif select == 2:
            model = load_cnn_model("network.h5")
            predict(model, training_path, test_path)
        elif select == 0:
            proceed = False
        else:
            print("\nInvalid option")

if __name__ == "__main__":

    print(" *** Animal Footprint Recognition started ***\n")
    menu()
    print("Finished")


# Unzip uploaded data

In [25]:
import zipfile
with zipfile.ZipFile('/content/Data.zip','r') as zip_ref:
     zip_ref.extractall("/")